In [59]:
import sys
import tensorflow as tf
import keras
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers
import os
import numpy as np
import pandas as pd

from keras import backend as K
from tqdm import tqdm

In [60]:
batch_size = 256
width = 50
height = 10
channels = 3
classes = 4
epochs = 100
dataset_dir = '/data/user/adipilat/ParticleID/genEvts/'
save_dir = '/data/user/adipilat/ParticleID/models/'
padding = 'padding' + str(height)
model_name= padding +'_model'
history_name = padding + '_history'

# This dictionary should be extended to new classes and antiparticles
class_labels = {22:0, 11:1, 13:2, 211:3}

In [61]:
# arrays of data needed for training

data_array = []
pid_array = []
en_array = []

# read dataset
files = [f for f in os.listdir(dataset_dir) if f.endswith("h5")]

for name in tqdm(files):
    print("Reading file", name)
    data = pd.read_hdf(dataset_dir + name)
    n_events = int(0.9 * data.event.max()) # using 90% of events for training

    for i in range(1, int(n_events+1)):
        tracksters = data.loc[(data['event'] == float(i)) & (data['trackster'] != float(0))]
        n_tracksters = tracksters.trackster.max()
        lead_en = 0
        for j in range(1, int(n_tracksters)+1):
            layerclusters = tracksters.loc[tracksters['trackster'] == float(j)]
            en = np.sum(layerclusters["E"].values)
            if(en>lead_en): #### since I shooted a single particle, only the leading trackster is considered
                lead_en = en
                image = np.zeros(width*height*channels).reshape(width,height,channels)
                pid = int(layerclusters["pid"].iloc[0])
                pid = class_labels[pid]
                en_value = layerclusters["genE"].iloc[0]
                for k in range(1, width+1):
                    layer = layerclusters[layerclusters['layer'] == float(k)]
                    if(len(layer) != 0):
                        temp = layer.E.values, layer.eta.values, layer.phi.values
                        temp = np.array(temp).T
                        dim = min(temp.shape[0],height)
                        image[k-1][:dim] = temp[:dim]
        data_array.append(image)
        pid_array.append(pid)
        en_array.append(en_value)    

    print("File", name, " processed")

data_array = np.array(data_array)
pid_array = np.array(pid_array)
pid_array = keras.utils.to_categorical(pid_array, num_classes=classes, dtype='float32')
en_array = np.array(en_array)


  0%|          | 0/4 [00:00<?, ?it/s]

Reading file gamma.h5



 25%|██▌       | 1/4 [11:01<33:04, 661.58s/it]

File gamma.h5  processed
Reading file electron.h5



 50%|█████     | 2/4 [21:59<22:00, 660.44s/it]

File electron.h5  processed
Reading file muon.h5



 75%|███████▌  | 3/4 [25:42<08:49, 529.28s/it]

File muon.h5  processed
Reading file pion_c.h5



100%|██████████| 4/4 [36:57<00:00, 573.00s/it]

File pion_c.h5  processed


In [62]:
print(data_array.shape)
print(en_array.shape)
print(pid_array.shape)

(36000, 50, 10, 3)
(36000,)
(36000, 4)


In [63]:
####### NORMALIZE THE ENERGY ########

mean_en = np.mean(en_array)
std_en = np.std(en_array)
print('Mean Energy Value: {}'.format(mean_en))
print('Std Energy Value: {}'.format(std_en))

en_array_norm = (en_array - mean_en)/std_en

Mean Energy Value: 213.90352475881576
Std Energy Value: 108.05413626100672


In [64]:
print('Creating model...')

def tree_model():

    input_img = Input(shape=(width, height, channels), name='input')
    
    conv = Conv2D(3, (5,1), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv1')(input_img)
    conv = Conv2D(3, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv2')(conv)
    conv = Conv2D(3, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv3')(conv)

    flat = Flatten()(conv)

    dense = Dense(1024, activation='relu', kernel_initializer='random_uniform', name='dense1')(flat)
    dense = Dense(256, activation='relu', kernel_initializer='random_uniform', name='dense2')(dense)

    dense_id = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_id1')(dense)
    dense_id = Dense(16, activation='relu', kernel_initializer='random_uniform', name='dense_id2')(dense_id)
    pid = Dense(classes, activation='softmax', kernel_initializer='random_uniform', name='pid_output')(dense_id)

    dense_er = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_er1')(dense)
    dense_er = Dense(8, activation='relu', kernel_initializer='random_uniform', name='dense_er2')(dense_er)
    enreg = Dense(1, name='enreg_output')(dense_er)

    model = Model(inputs=input_img, outputs=[pid, enreg])

    model.compile(loss={'pid_output': 'categorical_crossentropy', 'enreg_output': 'mse'}, loss_weights={'pid_output': 1, 'enreg_output': 2}, optimizer='adam', metrics={'pid_output': 'accuracy', 'enreg_output': 'mse'})
    return model

model = tree_model()
model.summary()

print("Model created!")

Creating model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 50, 10, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 10, 3)    48          input[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 50, 10, 3)    84          conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 50, 10, 3)    84          conv2[0][0]                      
___________________________________________________________________________________________

In [65]:
history = model.fit(data_array, {'pid_output': pid_array, 'enreg_output': en_array_norm}, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)], shuffle=True, verbose=1)
history_save = pd.DataFrame(history.history).to_hdf(save_dir + history_name + ".h5", "history", append=False)

Train on 32400 samples, validate on 3600 samples
Epoch 1/100
32400/32400 [==============================] - 2s 53us/step - loss: 2.4003 - pid_output_loss: 1.0522 - enreg_output_loss: 0.6741 - pid_output_acc: 0.4672 - enreg_output_mean_squared_error: 0.6741 - val_loss: 0.8866 - val_pid_output_loss: 0.2756 - val_enreg_output_loss: 0.3055 - val_pid_output_acc: 0.9186 - val_enreg_output_mean_squared_error: 0.3055
Epoch 2/100
32400/32400 [==============================] - 1s 30us/step - loss: 1.2497 - pid_output_loss: 0.4436 - enreg_output_loss: 0.4030 - pid_output_acc: 0.7250 - enreg_output_mean_squared_error: 0.4030 - val_loss: 0.7152 - val_pid_output_loss: 0.2055 - val_enreg_output_loss: 0.2549 - val_pid_output_acc: 0.9400 - val_enreg_output_mean_squared_error: 0.2549
Epoch 3/100
32400/32400 [==============================] - 1s 29us/step - loss: 1.1581 - pid_output_loss: 0.4240 - enreg_output_loss: 0.3670 - pid_output_acc: 0.7481 - enreg_output_mean_squared_error: 0.3670 - val_loss: 0.7

In [66]:
# Save model and weights
model.save(save_dir + model_name + ".h5")
print('Saved trained model at %s ' % save_dir)

Saved trained model at /data/user/adipilat/ParticleID/models/ 


In [67]:
# save the frozen model
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, save_dir, model_name + ".pbtxt", as_text=True)
tf.train.write_graph(frozen_graph, save_dir, model_name + ".pb", as_text=False)
print('Model saved')

INFO:tensorflow:Froze 508 variables.
INFO:tensorflow:Converted 508 variables to const ops.
Model saved


In [68]:
print("Done!")

Done!
